In [117]:
import pandas as pd
import json
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from multiprocessing import Process, Pool
from threading import Thread
import time
import timeit
import numpy as np
%matplotlib inline

# Nacteni dat

In [118]:
dataset = "movielens_1m"

In [119]:
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/items.json",'r') as f:
    items = json.loads(f.read())

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/properties.json",'r') as f:
    properties = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/user.folds.json",'r') as f:
    user_folds = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/users.int2str.json",'r') as f:
    users_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/users.str2int.json",'r') as f:
    users_str2int = json.loads(f.read())
      
ratings = pd.read_csv("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/ratings.csv",dtype = {'rating': np.float, 'itemId': np.str, 'userId':np.str} )

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/items.int2str.json",'r') as f:
    items_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/"+dataset+"/items.str2int.json",'r') as f:
    items_str2int = json.loads(f.read())

# Faktorizace

In [120]:
import multiprocessing
import ctypes
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as sparse_lg
import datetime
from math import sqrt

In [121]:
import numpy as np
from numpy.linalg import inv
# %load_ext line_profiler

In [159]:
class MatrixFactorization:
    def __init__(self, ratings, no_factors, relevant = .25, testset_amount = 0.2):
        self.Dataset = ratings
        self.Ratings = ratings.pivot_table(columns=['userId'],index=['itemId'],values='rating')
        self.Ratings_matrix = self.Ratings.values
        
        self.Testset = self.init_testset(testset_amount, relevant)
        
        self.Idx = np.isfinite(self.Ratings_matrix)
        self.Weights = np.isfinite(self.Ratings_matrix).astype(np.float64, copy=False)

        self.Ratings_matrix = np.nan_to_num(self.Ratings_matrix)
        self.Ratings_matrix_sparse = sp.csr_matrix(self.Ratings_matrix)
#         self.ratings_dict = dict(ratings.apply(lambda r:( (str(int(r.userId)),str(int(r.itemId))),{
#                                                                     'rating':r.rating,
#                                                                      }), axis=1).values)
        self.relevant = relevant
        self.number_of_obs_relevant_items = dict([(i,1+np.sum(self.Ratings_matrix[i,:] >= relevant)) for i in range(self.Ratings_matrix.shape[0]) ])

        self.testset_amount = testset_amount
        self.no_factors = no_factors
        self.Users = None
        self.Items = None
      
    def init(self):
        if(self.random_init): 
            user_shared_array_base = multiprocessing.Array(ctypes.c_double, np.random.rand(self.Ratings.shape[1]* self.no_factors), lock=False)
            user_shared_array = np.frombuffer(user_shared_array_base,dtype=float)    
            self.Users = user_shared_array.reshape(self.Ratings.shape[1], self.no_factors)
            
            
            item_shared_array_base = multiprocessing.Array(ctypes.c_double, np.random.rand(self.Ratings.shape[0] * self.no_factors), lock= False)
            item_shared_array = np.frombuffer(item_shared_array_base,dtype=float)
            self.Items = item_shared_array.reshape(self.Ratings.shape[0], self.no_factors)
            
    def init_optimizer(self):
        if(self.weights_mode == "AllRank"):
            self.Weights[self.Idx==False] = self.weight
            self.Weights_sparse = sp.csr_matrix(self.Weights)
            print("** Set weight: ",self.weight, " to missing ratings **")

        elif(self.weights_mode == "AllRank-pop"):
            pass
        
        elif(self.weights_mode == "MF-RMSE"):
            assert self.weight == 0, "Weight of missiong values MF-RMSE mode must be 0!"
            self.Weights_sparse = sp.csr_matrix(self.Weights)
            
        if(self.imputation_value != 0 ):   
            self.Ratings_matrix[self.Idx==False] = self.imputation_value
            self.Ratings_matrix_sparse = sp.csr_matrix(self.Ratings_matrix)
            print("** Surrogate missing rating values by imputation value: ", self.imputation_value, " **")
            
        
    def init_testset(self, testset_amount, relevant):
        ratings = self.Ratings_matrix
        Testset = {}
        
        for i_user in range(ratings.shape[1]):
            idx = np.where(ratings[:,i_user] >= relevant)[0]
            if(testset_amount):
                if(len(idx)):
                    random_choice = np.random.choice( idx, size = int(np.ceil(len(idx)*testset_amount)), replace = False)            
                    Testset[i_user] = {"indices": random_choice, "ratings": ratings[random_choice,i_user]}
                    ratings[random_choice,i_user] = np.nan
            else:
                Testset[i_user] = {"indices": idx, "ratings": ratings[idx,i_user]}
                
        return Testset
    '''
    ATOP
    '''
    
    def NRANKs_u(self, user, items):
        prediction = self.R_hat[:,user]
        ranks = np.argsort(prediction)
        N = len(prediction)
        nranks = np.array([np.where(ranks==item)[0][0] for item in items])/N
        return nranks

   
    
    def ATOP(self):
        atop = []
        RMSE = []
        for user, cont_id_rat in self.Testset.items():
            RMSE.extend(self.R_hat[cont_id_rat['indices'], user] - cont_id_rat['ratings'])
            nranks = self.NRANKs_u(user, cont_id_rat['indices'])
            atop.extend(nranks)
        
        return np.mean(atop), sqrt(np.mean((np.array(RMSE)**2)))
            
    
    def RECALL_u(self, u, K, beta):
        prediction = self.R_hat[:,u]
        T = self.Test_matrix
        idx = np.argsort(prediction)[::-1][:K]
        idx_relevant = np.where(T[:,u] == True)
    
        return np.sum(T[idx,u])/np.sum(T[:,u]), np.sum([1/(self.number_of_obs_relevant_items[i]**beta) for i in idx_relevant[0]]) #TODO IndexError: index 7591 is out of bounds for axis 0 with size 717


    
    def RECALL(self, K, beta):
        recall = 0
        for u in range(len(self.Users)):
            r, w = self.RECALL_u(u, K, beta)
            recall += r*w
        return recall
             
        
    '''
    RMSE
    '''  
    def RMSE(self, R, U, V, W):
        self.R_hat = np.dot(V,U.T)

        E = (R - (self.imputation_value + self.R_hat))
#         W_ = W[Idx] - self.weight
        return sqrt(((MFact.Weights==1).astype(np.float) * (E**2)).sum()/np.sum(W[W==1]) )
    
    '''
    OPTIMAZE PER FACTOR
    '''
    
    def compute_item_factor(self, batch):
        W, R, V = self.Weights, self.Ratings_matrix, self.Items
        lambda_, alfa, r_m = self.lambda_, self.alfa, self.imputation_value
        if(self.sparse):
            U = sp.csr_matrix(self.Users)
        else:
            U = self.Users


        if(self.opt_method == "solver"):
            if(self.sparse):
                for i in batch:
#                     print(i)
                    lM = sp.csr_matrix(R[i,:] - r_m).dot(sp.diags(W[i], 0)).dot(U)
                    rM = (U.T.dot(sp.diags(W[i], 0))).dot(U) + lambda_ * W[i].sum() * sp.eye(self.no_factors)
#                     V[i,:] = lM.dot(sparse_lg.inv(rM)).toarray()[0]
                    V[i,:] = sparse_lg.spsolve(rM,lM.T)
            elif(not self.sparse):
                for i in batch:
                    lM = np.dot(np.dot((R[i,:] - r_m),np.diag(W[i,:])),U)
                    rM = np.dot(np.dot(U.T,np.diag(W[i,:])),U) + lambda_ * W[i,:].sum() * np.eye(self.no_factors) #np.dot(np.trace(np.diag(W[i,:])),np.eye(self.no_factors))
                    V[i,:] = np.dot(lM, inv(rM))
#                     V[i,:] = np.linalg.solve(rM,lM)

        if(self.opt_method == "gradient-descent"):
            for i in batch:
                for c in range(50):
                    V[i] += -alfa * (np.dot((np.dot(V[i], U[Idx[i,:],:].T) - R[i,Idx[i,:]]), U[Idx[i,:],:]) + lambda_ * V[i]) #TODO pri AllRank je to spatne: Brat v uvahu vsechny hodnoceni 
        return
    
    def compute_user_factor(self, batch):
        W, R, U = self.Weights, self.Ratings_matrix, self.Users
        lambda_, alfa, r_m = self.lambda_, self.alfa, self.imputation_value
        if(self.sparse):
            V = sp.csr_matrix(self.Items)
        else:
            V = self.Items

        if(self.opt_method == "solver"):
            if(self.sparse):
                for u in batch:
                    lM = sp.csr_matrix(R[:,u].T - r_m).dot(sp.diags(W[:,u], 0)).dot(V)
                    rM = (V.T.dot(sp.diags(W[:,u], 0))).dot(V) + lambda_ * W[:,u].sum() * sp.eye(self.no_factors)
#                     U[u,:] = lM.dot(sparse_lg.inv(rM)).toarray()[0]
                    U[u,:] = sparse_lg.spsolve(rM,lM.T)
        
            elif(not self.sparse):
                for u in batch:
                    lM = np.dot(np.dot(R[:,u].T - r_m, np.diag(W[:,u])), V)
                    rM = np.dot(np.dot(V.T,np.diag(W[:,u])), V) + lambda_ * W[:,u].sum() * np.eye(self.no_factors) #np.dot(np.trace(np.diag(W[:,u]))
                    U[u,:] = np.dot(lM, inv(rM))
#                     U[u,:] = np.linalg.solve(rM,lM)

        if(self.opt_method == "gradient-descent"):
             for u in batch:
                for c in range(50):
                    U[u] += -alfa * (np.dot((np.dot(U[u], V[Idx[:,u],:].T) - R[Idx[:,u], u].T), V[Idx[:,u],:]) + lambda_* U[u] ) #TODO pri AllRank je to spatne: Brat v uvahu vsechny hodnoceni 
    
        return
    '''
    OPTIMIZE RMSE
    '''
    def optimize_rmse(self):
        self.init()
        self.init_optimizer()
        
        weighted_errors = []

        step_item = int(np.ceil(len(self.Items)/float(self.no_process)))
        step_user = int(np.ceil(len(self.Users)/float(self.no_process)))
        
        item_range = [range(i,min(len(self.Items), i+step_item)) for i in range(0, len(self.Items), step_item)]
        user_range = [range(i,min(len(self.Users), i+step_user)) for i in range(0, len(self.Users), step_user)]
        print("Item range", item_range)
        print("User range", user_range)
        
        for ii in range(self.no_iterations):
            if(not ii % 2):
                weighted_errors.append(self.RMSE(self.Ratings_matrix, self.Users, self.Items, self.Weights))
                print('{}th iteration is completed'.format(ii))
                print("RMSE (TRAIN) ", weighted_errors[-1])
                start_time = datetime.datetime.now()
                atop, rmse = self.ATOP()
                print("ATOP ", atop, " RMSE (TEST) ", rmse, " time ",datetime.datetime.now() - start_time)

            #ITEMS                
            process = []
            for batch in item_range:
                p = Process(target = self.compute_item_factor, args = (batch,))
                p.daemon = True

                process.append(p)
                p.start()
            
            [p.join() for p in process]
                
            #USERS
            process = []
            for batch in user_range:
                p = Process(target = self.compute_user_factor, args = (batch,))
                p.daemon = True

                process.append(p)
                p.start()
            
            [p.join() for p in process]

        return weighted_errors

        
    def optimaze(self, no_iterations, loss_function="RMSE", opt_method="solver",  lambda_ = 0.001, alfa=0.00001, 
                 no_process = 1, random_init = True, weights_mode = "MF-RMSE", weight = 0, imputation_value = 0, sparse = False, beta = 0, K = 10):
        
        self.loss_function = loss_function
        self.solve_opt_method = opt_method 
        self.lambda_ = lambda_
        self.alfa = alfa
        self.no_process = no_process
        self.no_iterations = no_iterations
        self.random_init = random_init
        self.weights_mode = weights_mode
        self.weight = weight
        self.imputation_value = imputation_value
        self.opt_method = opt_method
        self.sparse = sparse
        self.beta = beta 
        self.K = K
        
        weighted_errors = self.optimize_rmse()
        
        self.plot_rmse(weighted_errors)
        
    '''
    PLOT AND EXPLORE
    '''
    def plot_rmse(self, weighted_errors):
        plt.plot(np.log(weighted_errors), label="weighted error: "+str(weighted_errors[-1]))
        plt.ylabel("RMSE log scale")
        plt.xlabel("no iterations")

        plt.legend()
        plt.show()
        
    def explore(self):
        relevant = self.relevant 
        R = self.Ratings_matrix
        no_ratings = np.sum(self.Idx==True)
        no_missing = np.sum(self.Idx==False)
        no_all = no_ratings + no_missing
        
        sizes = [no_ratings, no_missing]
        labels = ["ratings", "missings"]
        colors = ['yellowgreen', 'lightskyblue']

        plt.pie(sizes, labels=labels,
                autopct='%1.1f%%', shadow=True, startangle=90, colors= colors)
        plt.axis('equal')
        plt.show()
        print("#{} rating, #{} missing".format(no_ratings, no_missing))
        print("Estimate offset w(m): ", no_ratings/no_missing)
        print("Avg of ratings: ", R[self.Idx].mean())
        
        no_relevant = np.sum(R[self.Idx]>=relevant)
        no_irelevant = np.sum(R[self.Idx]<relevant)
        labels = ["relevant", "irelevenat"]

        sizes = [no_relevant, no_irelevant]
        colors = ['lightskyblue', 'lightcoral']
        plt.pie(sizes, labels=labels,
                autopct='%1.1f%%', shadow=True, startangle=90, colors= colors)
        plt.axis('equal')
        plt.show()
        
        print("#{} relevant, #{} irelevant".format(no_relevant, no_irelevant))

        items_popularity = np.sort(list(MFact.number_of_obs_relevant_items.values()))[::-1]
#         plt.bar(range(len(items_popularity)),items_popularity)
        plt.xticks([])
        plt.ylabel("# of item rating marked as relevant")
        plt.xlabel("item")
        plt.plot(items_popularity,'_')
        plt.show()
        
        plt.hist(MFact.Dataset['rating'].values,bins = len(set(MFact.Dataset['rating'])))
        plt.xlabel("rating")
        plt.show()

In [160]:
# %lprun -f MFact.rmse_optimalization MFact.rmse_optimalization(0.001, 0.0001, 5)

In [161]:
ifold = 0
dataset = ratings[~ratings.userId.isin(user_folds[ifold])]
MFact = MatrixFactorization(dataset[:].copy(), 50, relevant = 1., testset_amount = 0.2)
for  i in range(MFact.Weights.shape[0]):
    if(not np.sum(MFact.Weights[i,:])):
        print(np.sum(MFact.Weights[i,:]))

In [162]:
# MFact.explore()

In [ ]:
start_time = datetime.datetime.now()
MFact.optimaze(10, opt_method="solver",  lambda_ = 0.05, no_process = 7, weights_mode = "AllRank", weight = 0, imputation_value = 0 , random_init = True, sparse = True, beta= 0.0, K = 20)
datetime.datetime.now() - start_time

** Set weight:  0  to missing ratings **
Item range [range(0, 529), range(529, 1058), range(1058, 1587), range(1587, 2116), range(2116, 2645), range(2645, 3174), range(3174, 3698)]
User range [range(0, 777), range(777, 1554), range(1554, 2331), range(2331, 3108), range(3108, 3885), range(3885, 4662), range(4662, 5436)]
0th iteration is completed
RMSE (TRAIN)  12.420342791186535
ATOP  0.496042724876  RMSE (TEST)  11.646013158934823  time  0:00:01.331495
2th iteration is completed
RMSE (TRAIN)  0.42101345097950804
ATOP  0.846635074581  RMSE (TEST)  0.6072545248814449  time  0:00:01.315831
4th iteration is completed
RMSE (TRAIN)  0.40462377406112776
ATOP  0.859710891103  RMSE (TEST)  0.5758469451535729  time  0:00:01.321525
6th iteration is completed
RMSE (TRAIN)  0.40025137404505523
ATOP  0.862734951392  RMSE (TEST)  0.5697966280444193  time  0:00:01.307996
8th iteration is completed
RMSE (TRAIN)  0.3983989163428437
ATOP  0.86368400697  RMSE (TEST)  0.568091891707482  time  0:00:01.31823